In today's class, you solved the exact same system of equations using LU as you did last week using row-reduction of an augmented matrix and back substitution. It certainly seems you had to do more work this time when you already had a way of solving equations. What is the advantage of using LU decomposition? This homework will explore this question.

> ## Make a copy of this notebook (File menu -> Make a Copy...)

Suppose that you are doing the same experiment on a number of different samples. You measure your outputs at the same time points, but get different results each time, depending on your sample. You want to fit polynomials to each of your data sets. As we saw when we fitted polynomials to data, this will involve solving an equation $Ax=b$ for different $b$'s, but always the same $A$.

Consider the following table, showing the results of three such experiments:

$t$ | 1 | 2 | 3 | 4 | 5 | 6 | 7
--- | :---: | :---: | :---: |:---: |:---: |:---: |:---: |
$y_1$ | 10 | 15 | -1 | 2 | -4 | 5 | 10
$y_2$ | 10 | 13 | 0 | 2 | -3 | 5 | 11
$y_3$ | 11 | 14 | -1 | 3 | -5 | 4 | 9

We will fit polynomials to each of these and compare using row-reduction and back-substitution to LU decomposition. To do so, we'll need to do the following:
* Understand the role of pivoting.<br><br>
* Solve the equations using row-reduction and back-substitution.<br><br>
* Lastly, compare this to solving them using LU decomposition, followed by forward- and back-substitution.

## Pivoting

Recall that LU decomposition with pivoting takes a matrix $A$ and returns matrices $P$, $L$, and $U$ so that $$PA=LU$$

We are trying to solve $Ax=v$. If we have a matrix $P$, then we can multiply both sides by it to get $$PAx=Pv$$ But $PA=LU$, so this is equivalent to $$LUx=Pv.$$

So all we need to do is multiply our $v$ by $P$ before we begin foward- or back-substitutions! Remember that each row in an augmented matrix represents one of the equations in the system. So all we are really doing here is swapping around the equations.

### Homework Question 1 

Use your row-reduction code and your `backsub(U,v)` function to find the coefficients of a sixth degree polynomial that fits each of the above data sets. In each case, this will involve solving $Ax=b$. 
1. Explain why the matrix $A$ is the same in each case. What is it?<br><br>
1. Write down the sixth degree polynomials in each case. Write the coefficients of each power to two decimal places.


</p> **Answer to Question 1.1** The matrix A does not change for the different cases because it is the polynomial equation which is set to fit the whole system, and is not changed by the distinct values of 'b'. For a polynomial the matrix A is a formal representation of (x^y, x^(y-1),x^(y-2), x^(y-3),... x^1, 1). We subsequently solve to get a set of coefficients which fit this matrix, by the method of RREF and backsubstitution outlined in the question <p>


In [126]:


import numpy as np
#function to swap two rows given matrix and indices
def swaprows(As,i,j):
    As[[i,j]] = As[[j,i]] #swap rows
    return As

#function to add mutliple (c) of row i to row j of matrix A
def rowaddmult(Am,i,j,c):
    Am[j] += c * Am[i] #perform row replacement
    return Am
#function performs the row reduction with pivoting to produce an upper triangular matrix
def rowredpivot(Ax):
    r,c = Ax.shape
    for x in range(r):
        if x < r-1:
            pivot = np.argmax(np.absolute(Ax[x:,x]))+x
            if type(pivot)==type(Ax):
                pivot = pivot[0]
            swaprows(Ax,x,pivot)
           
        for y in range(x+1, r):
            c = Ax[y,x]/Ax[x,x]
            rowaddmult(Ax, x, y, -c)
    return Ax

#function backsubstitutes to return an answer to v
def backsub(U, v):
    r,c = U.shape
    for x in reversed(range(0,r)):
        # use the dot product method of back substitution
        v[x] -= v[x+1:r]@U[x,x+1:c]
        # make value equal to one
        v[x] /= U[x,x]
    return v

# #Main body of new code
a = np.array([10,15,-1,2,-4,5,10]).astype(float)
b = np.array([10,13,0,2,-3,5,11]).astype(float)
c = np.array([11,14,-1,3,-5,4,9]).astype(float)

x = np.array([1,2,3,4,5,6,7]).astype('float')

A = np.ones((7,8)).astype('float')
B = A.copy()
C = A.copy()

A[:,7] = a
B[:,7] = b
C[:,7] = c

for i in range(6):
    for j in range(6-i):
        A[:,i] *= x
        B[:,i] *= x
        C[:,i] *= x


rowredpivot(A)
rowredpivot(B)
rowredpivot(C)


va = A[:,-1].copy()
backsub(A[:,0:-1],va)
vb = B[:,-1].copy()
backsub(B[:,0:-1], vb)
vc = C[:,-1].copy()
backsub(C[:,0:-1],vc)


print("Polynomial for y1 = " + polyPrinter(va))
print("Polynomial for y2 = " + polyPrinter(vb))
print("Polynomial for y3 = " + polyPrinter(vc))

Polynomial for y1 = -0.30x^6 + 7.27x^5 + -70.09x^4 + 339.48x^3 + -859.61x^2 + 1052.25x^1 + -459.00x^0
Polynomial for y2 = -0.24x^6 + 5.81x^5 + -55.89x^4 + 270.12x^3 + -682.37x^2 + 832.57x^1 + -360.00x^0
Polynomial for y3 = -0.33x^6 + 8.01x^5 + -76.68x^4 + 368.12x^3 + -921.99x^2 + 1114.87x^1 + -481.00x^0


### Homework Question 2
Write a function called `LUSolve(L,U,P,v)` that does the following given an LU decomposition of a matrix $A$:
1. First, multiplies the vector $v$ by $P$, as we discussed was needed.<br><br>
1. Solves $Ly=Pv$ by forward substition.<br><br>
1. Lastly, solves $Ux=y$ to find the solution of $Ax=v$.

Test your function on the data above. 

In [130]:
def fwdsub(L, v):
    r,c = U.shape
    for x in range(0,r):
        # use the dot product method of back substitution
        v[x] -= v[x-1:r]@L[x,x-1:c]
        # make value equal to one
        v[x] /= L[x,x]
        

        
def LU(Ax):
    U = Ax.copy()
    L = np.zeros_like(Ax)
    P = L.copy()
    np.fill_diagonal(P, 1.)
    r,c = U.shape
    for x in range(r):
        if x < r-1:
            pivot = np.argmax(np.absolute(Ax[x:,x]))+x
            if type(pivot)==type(Ax):
                pivot = pivot[0]
            swaprows(U,x,pivot)
            swaprows(L,x,pivot)
            swaprows(P,x,pivot)
        for y in range(x+1, r):
            c = U[y,x]/U[x,x]
            rowaddmult(U, x, y, -c)
            L[y,x] = c
    np.fill_diagonal(L, 1.)
    return (L,U,P)

A = np.ones((7,7)).astype('float')
for i in range(6):
    for j in range(6-i):
        A[:,i] *= x

L,U,P = LU(A)



def LUSolve(L, U, P, v):
    #Step 1
    v = P@v
    #Step 2
    y = fwdsub(L, v)
    print
    #Step 3
    backsub(U, v)
    return v

print(np.allclose(P@A,L@U))
print(LUSolve(L, U, P, a))
print(LUSolve(L, U, P, b))
print(LUSolve(L, U, P, c))



True
[-8.68402338e-03  1.99194213e-01 -1.77573403e+00  7.88939982e+00
 -1.82379791e+01  2.05016617e+01 -8.56785863e+00]
[-9.55242572e-03  2.19113635e-01 -1.95330744e+00  8.67833981e+00
 -2.00617770e+01  2.25518279e+01 -9.42464449e+00]
[-7.81562104e-03  1.79274792e-01 -1.59816063e+00  7.10045984e+00
 -1.64141812e+01  1.84514956e+01 -7.71107277e+00]


Note that since the matrix $A$ is always the same, we only have to use our $LU$ decomposition code once! This is much faster than having to do the row-reduction over and over for each output vector. The LU decomposition encodes the process of row-reduction in the lower-triangular matrix $L$, thus avoiding the need to recompute it.

Lastly, if you look at the data sets given above, you may notice that they are all quite similar to each other numerically. Yet the polynomials you generated are rather vastly different from each other. This is a serious problem. We say that the polynomial model has high *variance*. We will study this further in future labs.

### Homework Question 3

Write code that takes a set of *n* times (as a vector) and the outcomes of a number (say, *m*) of different experiments with measurements at those times (as an $m\times n$ array), and returns the coefficients of polynomials that fit each set of measurements. Your code should use LU decomposition and your `LUSolve(L,U,P,v)` function to make it as efficient as possible. Test your code on the above data.

In [136]:
def PolyBuilder(times, outcomes):
    sz = times.size
    r,c = outcomes.shape
   
    x = times.copy().astype('float')
    A = np.ones((sz,sz)).astype('float')

    for i in range(sz):
        for j in range(sz-i):
            A[:,i] *= x
           
    L,U,P = LU(A)        
           
    for s in range(r):
        vs = LUSolve(L,U,P,outcomes[s])
        print(vs)
       
times = np.array([1,2,3,4,5,6,7])
outcomes = np.array([[10,15,-1,2,-4,5,10],[10,13,0,2,-3,5,11],[11,14,-1,3,-5,4,9]])

print("Test Results: \n")
PolyBuilder(times,outcomes)

Test Results: 

[-1.24057477e-03  2.84563162e-02 -2.53676291e-01  1.12705712e+00
 -2.60542558e+00  2.92880882e+00 -1.22397980e+00]
[-1.36463225e-03  3.13019478e-02 -2.79043920e-01  1.23976283e+00
 -2.86596814e+00  3.22168970e+00 -1.34637778e+00]
[-1.11651729e-03  2.56106846e-02 -2.28308661e-01  1.01435141e+00
 -2.34488302e+00  2.63592794e+00 -1.10158182e+00]


### Homework Question 4 

Suppose you have a number of different output vectors $\vec{c}$ for the same set of equations. We have two different ways of solving $A\vec{x}=\vec{c}$:

* Row reduce the augmented matrix $[A|\vec{c}]$, then back substitute. Repeat for every different $\vec{c}$.<br><br>

* Find the $PA=LU$ decomposition of $A$, then use our `LUSolve(L,U,P,v)` function we wrote above.

Explain why we expect the second method to be far more efficient than the first if we have many different output vectors.

</p>**Answer:** The second method is far more efficient as the first when we have many different outcomes because the PA=LU decomposition method saves computation. Specifically, the row reduction method has a runtime of O(n^3) every time the algorithm is used. In contrast, the PA=LU decomposition method only has a runtime of O(n^3) the **first** time it runs to solve for P,L,U. After these values are calculated, the runtime to solve the system is only O(n^2).<p>

### Homework Question 5

Let's examine once again the *LU* decomposition of the matrix from the last homework: $$A=\begin{bmatrix} 10^{-4} & 0 & 10^4 \\ 10^4 & 10^{-4} & 0 \\ 0 & 10^4 & 1\end{bmatrix}.$$

As you saw in the lab, the code for *LU* decomposition without pivoting results in matrices *L* and *U* such that $A\neq LU$.

* By looking back at Question 4 from the lab and the work you did on floating point errors on Homework 3, explain exactly why you get the incorrect result you saw.

* Compute by hand the *PA=LU* decomposition for this matrix. Do you still expect a floating point error to occur? Explain why in this case, we still get the right answer using our `LU(A)` code.

In [143]:
A = np.array([[0.0001, 0, 10000],[10000,0.0001,0],[0,10000,1]])
L,U,P = LU(A)
print(U)
print(L)
print(P)

[[1.e+04 1.e-04 0.e+00]
 [0.e+00 1.e+04 1.e+00]
 [0.e+00 0.e+00 1.e+04]]
[[ 1.e+00  0.e+00  0.e+00]
 [ 0.e+00  1.e+00  0.e+00]
 [ 1.e-08 -1.e-16  1.e+00]]
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


</p>**Question 5.1:** As in the questions from homework 3, computing the $PA = LU$ decomposition for this matrix exceeded the precision allotted by Pythons representation of floats, thus truncation happens. 

**Question 5.2:** If we computed this by hand we would not experience a floating point error because when calculating the LU decomposition we do not have the same memory constraints as Python does, so we are not forced to truncate the values, hence complete accuracy. At the end of our LU(A) decomposition, we had a value which went beyond the maximum number of significant digits allowed in Python, so this error occured. We can see this comparing the results from LU(A) seen above, to the hand calculated results, outlined below, where the bottom right value of the U matrix has been truncated, as it was 10^-16. 
<p>

<br><br>

$\begin{bmatrix}
10^{-4} & 0 & 10^4\\
10^4 & 10^{-4} & 0\\
0 & 10^4 & 1\\
\end{bmatrix}$
$\begin{bmatrix}
? & ? & ?\\
? & ? & ?\\
? & ? & ?\\
\end{bmatrix}$
$\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1\\
\end{bmatrix}$
<br><br>
$\begin{bmatrix}
10^4 & 10^{-4} & 0\\
10^{-4} & 0 & 10^4\\
0 & 10^4 & 1\\
\end{bmatrix}$
$\begin{bmatrix}
? & ? & ?\\
? & ? & ?\\
? & ? & ?\\
\end{bmatrix}$
$\begin{bmatrix}
0 & 1 & 0\\
1 & 0 & 0\\
0 & 0 & 1\\
\end{bmatrix}$
<br><br>

$\begin{bmatrix}
10^4 & 10^{-4} & 0\\
0 & -10^{-12} & 10^4\\
0 & 10^4 & 1\\
\end{bmatrix}$
$\begin{bmatrix}
1 & ? & ?\\
10^{-8} & ? & ?\\
0 & ? & ?\\
\end{bmatrix}$
$\begin{bmatrix}
0 & 1 & 0\\
1 & 0 & 0\\
0 & 0 & 1\\
\end{bmatrix}$
<br><br>

$\begin{bmatrix}
10^4 & 10^{-4} & 0\\
0 & 10^4 & 1\\
0 & -10^{-12} & 10^4\\
\end{bmatrix}$
$\begin{bmatrix}
1 & ? & ?\\
0 & ? & ?\\
10^{-8} & ? & ?\\
\end{bmatrix}$
$\begin{bmatrix}
0 & 1 & 0\\
0 & 0 & 1\\
1 & 0 & 0\\
\end{bmatrix}$
<br><br>

$\begin{bmatrix}
10^4 & 10^{-4} & 0\\
0 & 10^4 & 1\\
0 & 0 & 10^4 + 10^{-16}\\
\end{bmatrix}$
$\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
10^{-8} & -10^{-16} & 1\\
\end{bmatrix}$
$\begin{bmatrix}
0 & 1 & 0\\
0 & 0 & 1\\
1 & 0 & 0\\
\end{bmatrix}$
<br><br>

### Optional Bonus Question

Write code that takes the coefficients of a polynomial and prints the polynomial with the coefficients printed to two decimal places. You should research Python functions that help you.

In [144]:
def polyPrinter(coefficients):
    coefficients = np.around(coefficients, decimals=2)
    polynomial = ""
    for i in range(len(coefficients)):
        polynomial += "{0:.2f}".format(coefficients[i]) + "x^" + str(len(coefficients) - i - 1) + " + "
    return polynomial[0:-3]

polyPrinter(va)



'-0.30x^6 + 7.27x^5 + -70.09x^4 + 339.48x^3 + -859.61x^2 + 1052.25x^1 + -459.00x^0'